In [ ]:
import numpy as np

def generate_dataset(n, p):
    # Generate mean and covariance matrix
    mean = np.zeros(p)
    cov_matrix = np.random.rand(p, p)
    cov_matrix = np.dot(cov_matrix, cov_matrix.T)  # Ensure positive semi-definiteness

    # Generate features with dependencies
    X = np.random.multivariate_normal(mean, cov_matrix, n)

    # Generate alpha coefficients
    alpha = np.random.randn(p)

    # Define Gaussian kernel function
    def gaussian_kernel(x1, x2, sigma=1.0):
        return np.exp(-((x1 - x2) ** 2) / (2 * sigma ** 2))

    # Compute logits
    logits = np.zeros(n)
    for i in range(n):
        for j in range(p):
            kernel_sum = np.sum(gaussian_kernel(X[:, j], X[i, j]))
            logits[i] += alpha[j] * kernel_sum

    # Sample y from Bernoulli distribution with computed logits
    probabilities = 1 / (1 + np.exp(-logits))
    y = np.random.binomial(1, probabilities)

    return X, y